In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import plotly.plotly as py
import plotly.graph_objs as go

import requests

from utils import utils_v2
plt.rcParams['figure.figsize'] = (15, 7) 
%matplotlib inline
%load_ext autoreload
%autoreload

In [2]:
threshold_array = np.logspace(np.log10(0.0001),np.log10(0.0505),250)
threshold_array_pct = np.logspace(np.log10(0.01),np.log10(5.05),250)

In [3]:
data = pd.read_csv('coinbaseUSD_1-min_data_2014-12-01_to_2018-06-27.csv')

data['timestamp'] = pd.to_datetime(data['Timestamp'],unit='s')
data.drop('Timestamp',axis=1,inplace = True)
data['Date'] = data.timestamp.dt.date
data = data.set_index('timestamp');
#date = '2017'    
stock_series = data.loc['2014 Dec 2':'2018 Jun 26']['Weighted_Price']
  


In [4]:
N_DC_array = []

for value in threshold_array:
    DCA, CPA = utils_v2.get_events(stock_series,threshold = value)
    N_DC_array.append(len(DCA))

In [5]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
model = lm.fit(threshold_array_pct.reshape(250,1),np.array(N_DC_array).reshape(250,1))

predictions = lm.predict(threshold_array_pct.reshape(250,1))

lm.score(predictions,np.array(N_DC_array).reshape(250,1))

-7003276547.899328

In [6]:
trace1 = go.Scatter(
    x=threshold_array_pct,
    y=N_DC_array,
    mode='markers',
    line=dict(
        color='rgb(250,0,0)'
    )
)

trace2 = go.Scatter(
    x=threshold_array_pct,
    y=(threshold_array_pct.reshape(250,1)/5.430)**(-2.178),
    mode='markers',
    line=dict(
        color='rgb(0,250,0)'
    )
)

data = [trace1, trace2]

layout = go.Layout(
    xaxis=dict(
        type='log',
        autorange=True
    ),
    yaxis=dict(   
        type='log',
        autorange=True
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='text', fileopt='append')